# Download Email Events from Hubspot API

This app will download all email events from Hubspot API. It uses Campaign Statistics to decide what timeframe to begin downloading from.

Resources:
* MSSQL - Run DDL in the project against your database
* Schema named "hs" in database

Put the event types you want to download in the hs.DownloadableEventType table

In [1]:
import requests, json, time
import pandas as pd
import sqlalchemy
import pyodbc as odbc

import os
import configparser, json

import globvars
import functions
import campaigns
import events
import contacts

In [2]:
parser = configparser.ConfigParser()

if os.name == "posix":
    parser.read(u'../config.ini')
else:
    parser.read(u"..\config.ini")

In [3]:
key =  parser.get('Hubspot', 'APIKEY')
apikey = "?hapikey=" + key
limit = "&limit=1000"
firstparm = apikey + limit
baseurl = "https://api.hubapi.com"
runningcallcount = 0

In [4]:
# SQLAlchemy Connection
meta_engine = sqlalchemy.create_engine(parser.get('Database', 'HubspotSqlAlc'))

# Test the connection
connection = meta_engine.connect()
result = connection.execute("SELECT @@ServerName ServerName, DB_NAME() AS DatabaseName, CURRENT_USER AS LoggedInAs")
for row in result:
    print (row['ServerName'], row['DatabaseName'], row['LoggedInAs'])

(u'DESKTOP-R5USRDR', u'Hubspot', u'mac')


In [5]:
# PYODBC Connection
conn = odbc.connect(parser.get('Database', 'HubspotPy'), autocommit=True)
curs = conn.cursor()
curs.execute("SELECT @@ServerName AS ServerName, DB_NAME() AS DatabaseName, CURRENT_USER AS LoggedInAs")
print curs.fetchone()

(u'DESKTOP-R5USRDR', u'Hubspot', u'mac')


## Step 1: Setup the Download Run
I like to keep track of each run and when I did it

In [6]:
curs.execute("INSERT INTO hs.DownloadRun(RunDate) VALUES(GETDATE())")
ident = curs.execute("SELECT SCOPE_IDENTITY()").fetchone()[0]

## Step 2: Get New Campaign Stats

In [7]:
# Get All the campaign statistics
numCalls, campaignStats = campaigns.get_all_campaignsstats(baseurl, apikey, firstparm, noisy=True)

runningcallcount = runningcallcount + numCalls

# put them in database
pd.DataFrame.from_dict(campaignStats, orient='columns', dtype=None).to_sql(name="CampaignStatistics"
                               , schema="hs"
                               , con=meta_engine
                               , index=False
                               , if_exists="replace"
                               , dtype=globvars.COLUMNSCAMPAIGNSTATISTICS)

https://api.hubapi.com/email/public/v1/campaigns/by-id?hapikey=9bd03e6c-40a3-43f8-90b4-6e010f52a945&limit=1000
dumping file
Campaign List Call Count: 1
Number of Campaigns: 292


## Step 3: Loop through the events we want to capture and download

In [ ]:
# hs.DownloadableEventType stores the event Types I want
sql = "SELECT * FROM hs.DownloadableEventType"
eventList = pd.read_sql(sql, meta_engine)

for index, row in eventList.iterrows():
    events.get_new_events(baseurl, firstparm, curs, meta_engine, row['eventType'], ident)

## Step 4: Store Stats History
Finalize the data

In [ ]:
df = dictToDF(campaignStats)
df['RunID'] = ident

df.to_sql(name="CampaignStatisticsHistory"
            , schema="hs"
            , con=meta_engine
            , index=False
            , if_exists="append"
            , dtype=globvars.COLUMNSCAMPAIGNSTATISTICSHISTORY)